In [1]:
cd

/Users/emiljaffal


In [ ]:
import pandas as pd

file_path = "desktop/data.xlsx"
data = pd.read_excel(file_path)
data

In [ ]:
cd desktop/elemnet/elemnet

In [ ]:
#had to update composition.py, data.py, conversions.py from matminer... code not maintained in accordance to updated submodules:
    #from pymatgen.core.composition import Composition #UPDATE
    #from pymatgen.ext.matproj import MPRester

import re, numpy as np, os, sys, pandas
from pymatgen.core.composition import Composition 
import data_utils
import magpie
from matminer.featurizers.base import MultipleFeaturizer
from matminer.featurizers import composition as cf
from matminer.utils.conversions import str_to_composition
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [ ]:
formula_unitcell = data[['Formula', 'Unit cell volume, A^3']]
formula_unitcell.rename(columns={'Formula':'composition'}, inplace=True)
formula_unitcell

In [ ]:
formula_unitcell['comp_obj'] = formula_unitcell['composition'].apply(lambda x: Composition(x))
formula_unitcell

In [ ]:
formula_unitcell['pretty_comp'] = formula_unitcell['comp_obj'].apply(lambda x: x.reduced_formula)
formula_unitcell

In [ ]:
#formula_unitcell.drop_duplicates('pretty_comp', keep='first', inplace=True)
#print('Reduced dataset to %d entries'%len(formula_unitcell))

In [ ]:
formula_unitcell['nelems'] = formula_unitcell['comp_obj'].apply(lambda x: len(x))
formula_unitcell

In [ ]:
formula_unitcell.groupby('nelems').count()

In [ ]:
formula_unitcell['comp_dict'] = formula_unitcell['pretty_comp'].apply(lambda x: data_utils.parse_formula(x))
formula_unitcell

In [ ]:
elements_present = set()
for i, row in formula_unitcell.iterrows():
    elements_present |= set(row['comp_dict'].keys())
print (elements_present)

In [ ]:
elements = ['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 
            'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb',
            'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 
            'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta',
            'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th', 'Pa',
            'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt',
            'Ds', 'Rg', 'Cn']


elements_tl = ['H', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'K',
 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se',
 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In',
 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd',
 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au',
 'Hg', 'Tl', 'Pb', 'Bi', 'Ac','Th', 'Pa', 'U', 'Np', 'Pu']

elem_pos = dict()
i=0
for el in elements:
   elem_pos[el] = i
   i+=1

In [ ]:
print (elements_present, len(elements_present))
print ([e for e in elements_present if e not in elements_tl])
print ([e for e in elements_tl if e not in elements_present])

In [ ]:
formula_unitcell['comp_fractions'] = formula_unitcell['comp_dict'].apply(lambda x: data_utils.get_fractions(x))
formula_unitcell

In [ ]:
print (formula_unitcell[:1])
#oqmd_data['solution_term'] = oqmd_data['comp_obj'].apply(lambda x: compute_mixing_term(x))
print (formula_unitcell.shape)
formula_unitcell = formula_unitcell[~formula_unitcell['comp_fractions'].isnull()]
print (formula_unitcell.shape)

In [ ]:
for i,e in enumerate(elements_tl):
    formula_unitcell[e] = [ x[i] for x in formula_unitcell['comp_fractions']]
formula_unitcell.shape

In [ ]:
formula_unitcell.columns

In [ ]:
feature_calculators = MultipleFeaturizer([cf.Stoichiometry(), cf.IonProperty(fast=True)])

In [ ]:
#feature_calculators = MultipleFeaturizer([cf.Stoichiometry(), cf.ElementProperty.from_preset("magpie"),
                                         # cf.ValenceOrbital(props=['avg']), cf.IonProperty(fast=True)])

#Magpie folder is empty...

In [ ]:
feature_labels = feature_calculators.feature_labels()

In [ ]:
print (feature_labels, len(feature_labels)) #9/145 are not magpie-based

In [ ]:
feature_calculators.featurize_dataframe(formula_unitcell, col_id='comp_obj');
formula_unitcell